## Import Modules

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import torch
import re
import datasets
import evaluate
import accelerate

from datasets import load_dataset
from tensorflow import keras

from keras.layers import Embedding, LSTM, Dense, Bidirectional, SpatialDropout1D, BatchNormalization, Dropout
from keras.models import Sequential
from keras.optimizers.legacy import Adam
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical, pad_sequences

from nltk.corpus import stopwords
from gensim.models import Word2Vec

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

## Import the datasets

In [2]:
dataset = load_dataset("climatebert/climate_sentiment")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 320
    })
})

## Attention-Based for Text Classification

### Data pre-processing

In [26]:
text_train = dataset["train"]["text"]
label_train = dataset["train"]["label"]

text_test = dataset["test"]["text"]
label_test = dataset["test"]["label"]

In [27]:
TEST_SIZE = 0.2

_, train_set = dataset["train"].train_test_split(test_size=TEST_SIZE).values()
_, test_set = dataset["test"].train_test_split(test_size=TEST_SIZE).values()

mini_ds = datasets.DatasetDict(
    {
        "train": train_set,
        "test" : test_set
    }
)

In [28]:
mini_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 64
    })
})

### Prepare the datasets for RNN-Based Models (to compare with BERT-Based)

In [29]:
# define max words for the vocabulary
MAX_WORDS = 10000
tokenizer_train = Tokenizer(num_words=MAX_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer_test = Tokenizer(num_words=MAX_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

# fit dataset to tokenizer
tokenizer_train.fit_on_texts(text_train)
tokenizer_test.fit_on_texts(text_test)

# convert dataset to sequence of integer
seq_train = tokenizer_train.texts_to_sequences(text_train)
seq_test = tokenizer_test.texts_to_sequences(text_test)

# pad the sequence to fixed_length, will adjust later
MAX_SEQ = 250
X_train = pad_sequences(sequences=seq_train, maxlen=MAX_SEQ)
X_test = pad_sequences(sequences=seq_test, maxlen=MAX_SEQ)

# turn the lables into categorical
y_train = to_categorical(label_train, 3)
y_test = to_categorical(label_test, 3)

### Tokenize dataset for the BERT-Based model

In [31]:
# create tokenizer from BERT
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(ds):
    return tokenizer(ds["text"], padding="max_length", truncation=True)

# tokenize the dataset
tokenized_ds = mini_ds.map(tokenize, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

### Create RNN-Based Model

In [32]:
EMBEDDING_DIM = 128
w2v = Word2Vec(sentences=text_train, vector_size=EMBEDDING_DIM, window=5, min_count=1, sg=0)

In [33]:
embedding_matrix = np.zeros((MAX_WORDS, EMBEDDING_DIM))
for word, i in tokenizer_train.word_index.items():
    if i < MAX_WORDS:
        if word in w2v.wv:
            embedding_matrix[i] = w2v.wv[word]

In [34]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
# create sequential model to stack layers
rnn_w2v = Sequential()

# embedding layer to convert integer tokens into dense vectors
# change the weight to embedding_matrix from Word2Vec
rnn_w2v.add(Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=X_train.shape[1], weights=[embedding_matrix], trainable=True))

# performs variational dropout in NLP models
rnn_w2v.add(SpatialDropout1D(rate=0.2))

# Bidirectional LSTM layers with dropout and recurrent dropout
rnn_w2v.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
rnn_w2v.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2)))

# Batch normalization layer
rnn_w2v.add(BatchNormalization())

# Dense layer with ReLU activation
rnn_w2v.add(Dense(64, activation="relu"))

# Dropout layer
rnn_w2v.add(Dropout(0.5))

# add dense layer, with 3 output and softmax activation (used for multiclass)
rnn_w2v.add(Dense(3, activation="softmax"))

# Learning rate reduction on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the RNN model with custom optimizer (Adam) and loss function (categorical_crossentropy)
optimizer = Adam(learning_rate=0.001)
rnn_w2v.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [36]:
# train the model
rnn_w2v_history = rnn_w2v.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
32/32 [==============================] - 17s 423ms/step - loss: 1.0925 - accuracy: 0.4110 - val_loss: 1.0751 - val_accuracy: 0.5094
Epoch 2/20
32/32 [==============================] - 13s 404ms/step - loss: 0.8525 - accuracy: 0.6280 - val_loss: 1.0751 - val_accuracy: 0.4656
Epoch 3/20
32/32 [==============================] - 13s 408ms/step - loss: 0.5024 - accuracy: 0.8120 - val_loss: 1.1118 - val_accuracy: 0.4469
Epoch 4/20
32/32 [==============================] - 18s 552ms/step - loss: 0.2342 - accuracy: 0.9310 - val_loss: 1.1245 - val_accuracy: 0.4594
Epoch 5/20
32/32 [==============================] - 20s 638ms/step - loss: 0.1359 - accuracy: 0.9650 - val_loss: 1.2585 - val_accuracy: 0.4656
Epoch 6/20
32/32 [==============================] - 20s 631ms/step - loss: 0.0865 - accuracy: 0.9740 - val_loss: 1.3705 - val_accuracy: 0.4031
Epoch 7/20
32/32 [==============================] - 19s 591ms/step - loss: 0.0680 - accuracy: 0.9790 - val_loss: 1.9884 - val_accuracy: 0.3469

In [37]:
# evaluate the models
loss, accuracy = rnn_w2v.evaluate(X_test, y_test)
print(f"Loss:\t{loss:.4f}")
print(f"Accuracy:\t{accuracy:.4f}")

10/10 [==============================] - 1s 60ms/step - loss: 3.5275 - accuracy: 0.3281
Loss:	3.5275
Accuracy:	0.3281


### Create a BERT-Based model

In [38]:
bert_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=-1)
    return metric.compute(predictions=pred, references=labels)

In [20]:
bert_training_args = TrainingArguments(output_dir="trainer", evaluation_strategy="epoch")

bert_trainer = Trainer(
    model= bert_model,
    args= bert_training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics,
)

In [21]:
bert_trainer.train()

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 0.7842980623245239, 'eval_accuracy': 0.75, 'eval_runtime': 12.167, 'eval_samples_per_second': 5.26, 'eval_steps_per_second': 0.658, 'epoch': 1.0}


  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 0.7379715442657471, 'eval_accuracy': 0.6875, 'eval_runtime': 5.7386, 'eval_samples_per_second': 11.153, 'eval_steps_per_second': 1.394, 'epoch': 2.0}


  0%|          | 0/8 [00:00<?, ?it/s]

{'eval_loss': 0.7074193358421326, 'eval_accuracy': 0.71875, 'eval_runtime': 14.6874, 'eval_samples_per_second': 4.357, 'eval_steps_per_second': 0.545, 'epoch': 3.0}
{'train_runtime': 307.3582, 'train_samples_per_second': 1.952, 'train_steps_per_second': 0.244, 'train_loss': 0.6102455647786459, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=0.6102455647786459, metrics={'train_runtime': 307.3582, 'train_samples_per_second': 1.952, 'train_steps_per_second': 0.244, 'train_loss': 0.6102455647786459, 'epoch': 3.0})

In [22]:
bert_result = bert_trainer.evaluate()

  0%|          | 0/8 [00:00<?, ?it/s]

In [23]:
bert_result

{'eval_loss': 0.7074193358421326,
 'eval_accuracy': 0.71875,
 'eval_runtime': 41.3466,
 'eval_samples_per_second': 1.548,
 'eval_steps_per_second': 0.193,
 'epoch': 3.0}

### Comparing RNN-Based to BERT-Based

In [51]:
PREDICT_SIZE = 10

In [55]:
# using Word2Vec embedding
predictions = rnn_w2v.predict(X_test[:PREDICT_SIZE])
score = 0

for text, prediction, groundtruth in zip(tokenizer_test.sequences_to_texts(X_test), predictions, y_test[:5]):
    pred = prediction.tolist()
    pred = pred.index(max(pred))

    groundtruth = groundtruth.tolist()
    groundtruth = groundtruth.index(max(groundtruth))

    if (groundtruth == pred):
        score += 1

    print(f"Text: {text}\Groundtruth: {groundtruth}\nPredicted: {pred}\n")

print(f"Score: {score}/{PREDICT_SIZE}")

1/1 [==============================] - 1s 932ms/step
Text: sustainable strategy ‘red lines’ for our sustainable strategy range we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest performing companies from an esg perspective are not eligible for investment\Groundtruth: 0
Predicted: 2

Text: verizon’s environmental health and safety management system provides a framework for identifying controlling and reducing the risks associated with the environments in which we operate besides regular management system assessments internal and third party compliance audits and inspections are performed annually at hundreds of facilities worldwide the goal of these assessments is to identify and correct site specific issues and to educate and empower facility managers and supervisors to implement corrective actions verizon’s environment health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities\G

In [ ]:
result = bert_trainer.predict(tokenized_ds["test"])
score = 0

for i in range(PREDICT_SIZE):
    groundtruth = np.argmax(result[0][i])
    pred = tokenized_ds['test']['label'][i]

    if (groundtruth == pred):
        score += 1

    print(f"Text: {tokenized_ds['test']['text'][i]}")
    print(f"Groundtruth: {groundtruth}")
    print(f"Prediction: {pred}\n")

print(f"Score: {score}/{PREDICT_SIZE}")

  0%|          | 0/8 [00:00<?, ?it/s]

Text: We recognize the importance of effectively managing climate-related risks and opportunities and have embedded them into Verizon’s existing processes and decision making. That is why we are committed to strengthening our existing Board-level oversight and governance structures with regard to climate. This includes clarifying lines of communication between Verizon man- agement and the Board on climate- and ESG-related issues, generally creating continuous and frequent lines of communication on sustainability issues, and continuing to elevate the transparency of our ESG disclosures.
Groundtruth: 1
Prediction: 1

Text: The objectives of the CCWG, which reflect the risks and opportunities facing our business from climate change, are: • Build operational climate resilience to protect Newton’s own assets • Effectively assess, integrate, monitor and manage climate-related investment risks and opportunities to protect and grow our clients’ assets, supported by our climate change investmen